**装饰器学习笔记**  
@author: Rui Zhu  
@created: 2024-05-10  
@cite:  
1. 《流畅的Python》

---
# 装饰器的简单实例
* 装饰器的定义：装饰器是一种可调用对象，其参数是另一个函数（被装饰函数）
* 装饰器的行为：装饰器可能对被装饰函数做些处理，然后返回新函数或可调用对象
* 装饰器的3个基本性质:
    1. 装饰器是一个函数或其他可调用对象
    2. 装饰器可以把被装饰函数替换成别的函数
    3. 装饰器在加载模块时立即执行

In [8]:
def deco(func):
    """
    定义一个装饰器deco, 功能是输入任意函数, 返回新函数inner
    """
    print("装饰器会立即执行")
    def inner():
        print('running inner()')
    return inner

@deco  # 使用deco装饰器修饰target函数
def target():
    print('running target()')

装饰器会立即执行


In [10]:
target()  # 调用target函数, 实际上调用的是inner函数

running inner()


---
# 装饰器的重要性质: 装饰器会在被装饰函数定义后立即执行
* 即, 使用@调用装饰器相当于调用一次装饰器函数

In [13]:
registry = []  # 定义一个列表, 用于存储被装饰的函数
def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register  # 使用register装饰器修饰f1函数
def f1():
    print('running f1()')

@register  # 使用register装饰器修饰f2函数
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

running register(<function f1 at 0x107a99440>)
running register(<function f2 at 0x107a980e0>)


---
# Python中的变量作用域
* 因为装饰器涉及到嵌套函数, 所以需要弄清楚各种变量的作用范围

## 全局变量与局部变量

In [20]:
b = 6  # 全局变量b
def f1(a):
    print(a) # 局部变量a
    print(b)

f1(3)

3
6


In [86]:
b = 6  # 全局变量b
def f1(a):
    print(a) # 局部变量a
    print(b)
    b = 9

try:
    f1(3)  # 报错的原因是: python会假设函数主体中赋值的变量是局部变量
except Exception as e:
    print(e)

3
cannot access local variable 'b' where it is not associated with a value


In [44]:
b = 6  # 全局变量b
def f1(a):
    global b  # 手动生命b是全局变量
    print(a) # 局部变量a
    print(b)
    b = 9

f1(3)

3
6


## 闭包
* 见实例2, 闭包是一个函数, 包含原始函数和该函数定义时使用的自由变量的绑定
* 自由变量时外层函数的局域变量, 也是内层函数的绑定
* 调用闭包函数时, 即使局域作用域失效了, 也可以使用这些绑定

In [54]:
"""
实例1: 计算累计平均值的类实现
"""
class Averager:
    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

avg = Averager()
print(avg(10))
print(avg(11))
avg.series

10.0
10.5


[10, 11]

In [73]:
"""
实例2: 计算累计平均值的高阶函数实现
"""
def make_averager():

    # 闭包
    series = []  # make_averager函数的局部变量, 也是averager函数的自由变量
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    
    return averager

avg = make_averager()
print(avg(10))
print(avg(11))

avg.__closure__[0].cell_contents  # 查看自由变量series的值

10.0
10.5


[10, 11]

## nonlocal关键字声明自由变量

In [84]:
"""
实例2: 计算累计平均值的高阶函数实现, 不重复求和计算, 速度会更快
"""
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        count += 1  # count是数字或任何不可变类型, python会认为count是局部变量
        total += new_value
        return total / count

    return averager 

avg = make_averager()

try:
    avg(10)  # 报错的原因是: python会假设函数主体中赋值的变量是局部变量
except Exception as e:
    print(e)

cannot access local variable 'count' where it is not associated with a value


In [82]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total  # 声明count和total是自由变量可解决这个问题
        count += 1  
        total += new_value
        return total / count

    return averager 

avg = make_averager()
avg(10)
avg(11)

10.5

---
# 实现简单的装饰器